# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

2. Найти и вывести на экран названия всех альбомов группы Accept

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [1]:
import  sqlite3

In [2]:
con = sqlite3.connect("./resource/recipes.db")
cur = con.cursor()

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [3]:
# cur.execute('DROP TABLE Recipe')
# cur.execute('DROP TABLE Review')

In [4]:
cur.execute('''CREATE TABLE Recipe 
               (id INTEGER PRIMARY KEY NOT NULL, 
               name VARCHAR(200), 
               minutes INTEGER, 
               submitted VARCHAR(10), 
               description VARCHAR(1000), 
               n_ingredients INTEGER)''')

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [5]:
cur.execute('''CREATE TABLE Review 
               (id INTEGER PRIMARY KEY NOT NULL, 
               user_id INTEGER, 
               recipe_id INTEGER,
               date VARCHAR(10), 
               rating REAL, 
               review VARCHAR(1000),
               FOREIGN KEY (recipe_id) REFERENCES Recipe(id))''')

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [6]:
import csv

In [7]:
with open('../sem02/data/reviews_sample.csv') as fp2:
    reader2 = csv.reader(fp2)
    res = [row for row in reader2][:3]

In [8]:
with open('../sem05/result/recipes_sample_with_tags_ingredients.csv') as fp5:
    reader5 = csv.reader(fp5)
    res = [row for row in reader5][:3]

In [9]:
with open('../sem05/result/recipes_sample_with_tags_ingredients.csv') as fp5:
    reader5 = csv.reader(fp5)
    header = next(reader5) 
    for _, name, id, minutes, contributor_id, submitted, n_steps, description, n_ingredients, ingredients in reader5:
        cur.execute('INSERT INTO Recipe (id, name, minutes, submitted, description, n_ingredients) VALUES (?,?,?,?,?,?)', 
                (id, name, minutes, submitted, description, n_ingredients))
        
        
# проверка 
for row in cur.execute('SELECT * FROM Recipe LIMIT 0, 3'):
    print(row)

(48, 'boston cream pie', 135, '1999-08-24', '', 15)
(55, 'betty crocker s southwestern guacamole dip', 125, '1999-09-15', '', 5)
(66, 'black coffee barbecue sauce', 30, '1999-09-10', "it's great to know folks like this sauce so much! i wish i could take credit for it but it was adopted after the original member left. thank you, all, just the same.\ni have made the recipe with regular brewed coffee but highly recommend the espresso. in a pinch use strong instant if you have nothing else. the acid of the coffee balances the sweetness of the other ingredients.", 11)


In [10]:
with open('../sem02/data/reviews_sample.csv') as fp2:
    reader2 = csv.reader(fp2)
    header = next(reader2) 
    for _, user_id, recipe_id, date, rating, review in reader2:
        cur.execute('INSERT INTO Review (id, user_id, recipe_id, date, rating, review) VALUES (?,?,?,?,?,?)', 
                (_, user_id, recipe_id, date, rating, review))
        
        
# проверка
for row in cur.execute('SELECT * FROM Review LIMIT 0, 3'):
    print(row)

(5, 52282, 120345, '2005-05-21', 4.0, "very very sweet. after i waited the 2 days i bought 2 more pints of raspberries and added them to the mix. i'm going to add some as a cake filling today and will take a photo.")
(6, 124416, 120345, '2011-08-06', 0.0, 'Just an observation, so I will not rate.  I followed this procedure with strawberries instead of raspberries.  Perhaps this is the reason it did not work well.  Sorry to report that the strawberries I did in August were moldy in October.  They were stored in my downstairs fridge, which is very cold and infrequently opened.  Delicious and fresh-tasting prior to that, though.  So, keep a sharp eye on them.  Personally I would not keep them longer than a month.  This recipe also appears as #120345 posted in July 2009, which is when I tried it.  I also own the Edna Lewis cookbook in which this appears.')
(7, 2000192946, 120345, '2015-05-10', 2.0, 'This recipe was OVERLY too sweet.  I would start out with 1/3 or 1/4 cup of sugar and jsut 

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [11]:
seq = cur.execute('''SELECT * FROM Recipe 
                        WHERE n_ingredients = 10
                        LIMIT 0, 5''')

seq.fetchall()

[(246, 'lee s hot crab dip', 45, '1999-09-01', "lee's hot crab dip", 10),
 (289,
  'feijoada  brazilian bean soup  ii',
  150,
  '1999-10-04',
  'brazilian bean soup',
  10),
 (544, 'greek mushroom salad', 38, '1999-08-10', '', 10),
 (671, 'cucumber relish', 40, '1999-09-10', '', 10),
 (975,
  'basil parmesan biscuits',
  27,
  '1999-08-14',
  'perfect with pasta or other italian dishes',
  10)]

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [12]:
seq = cur.execute('''SELECT name FROM Recipe 
                        WHERE minutes IN 
                            (SELECT MAX(minutes) FROM Recipe) ''')

seq.fetchall()

[('strawberry liqueur',), ('blueberry liqueur',)]

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [13]:
def user_input():
    id_ = input("Введите id: ")
    try:
        id_ = int(id_)
        return id_
    except ValueError:
        print('Введите число типа int')
        return user_input()

In [14]:
id_ = user_input()
seq = cur.execute(f'SELECT * FROM Recipe WHERE id = {id_}',)  
rez = seq.fetchone()
if not rez:
    print(f'Рецепт с id = {id_} отсутствует')
else:
    print(rez)
    
# введите 57993

Введите id: 57993
(57993, 'salmon with tomatoes', 50, '2003-04-04', 'this savoury dish is very popular in andalusia, spain. you can use red pepper flakes (to your taste) in place of the hot pepper and any firm whitefish can be a substitute for the salmon. great with rice and a green vegetable.', 13)


8. Найдите кол-во отзывов с рейтингом 5.

In [15]:
seq = cur.execute('''SELECT COUNT(*) FROM Review
                        WHERE rating = 5''')

seq.fetchall()

[(91361,)]

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [16]:
seq = cur.execute('''SELECT COUNT(DISTINCT name) FROM Recipe
                        WHERE id IN
                            (SELECT recipe_id FROM Review
                                WHERE rating >= 4)''')

seq.fetchall()

[(26297,)]

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [17]:
seq = cur.execute('''SELECT COUNT(*) FROM Recipe
                        WHERE strftime('%Y', submitted) = '2010' AND
                        minutes >= 15''')

In [18]:
seq.fetchall()

[(1319,)]

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [19]:
seq = cur.execute('''SELECT rc.id, rc.name, rv.user_id, rv.date, rv.rating
                        FROM Recipe rc
                        INNER JOIN Review rv
                        ON rc.id = rv.recipe_id
                        WHERE rc.n_ingredients >= 3
                        ''')

seq.fetchall()

[(134728, 'kfc honey bbq strips', 76535, '2005-09-02', 4.0),
 (134728, 'kfc honey bbq strips', 273745, '2005-12-22', 5.0),
 (134728, 'kfc honey bbq strips', 190375, '2007-03-09', 5.0),
 (134728, 'kfc honey bbq strips', 468945, '2008-02-20', 0.0),
 (134728, 'kfc honey bbq strips', 255338, '2008-04-11', 5.0),
 (134728, 'kfc honey bbq strips', 1171894, '2009-04-21', 5.0),
 (200236,
  'lamb stew with tomatoes  chickpeas and spices',
  68960,
  '2006-12-19',
  4.0),
 (200236,
  'lamb stew with tomatoes  chickpeas and spices',
  618928,
  '2007-10-17',
  4.0),
 (200236,
  'lamb stew with tomatoes  chickpeas and spices',
  217118,
  '2008-04-18',
  5.0),
 (41090, 'black and white bean salad', 58332, '2003-02-25', 4.0),
 (145160, 'thanksgiving mashed potatoes', 263525, '2005-11-17', 5.0),
 (180132, 'chicken vegetable soup low carb low fat', 58439, '2007-06-09', 4.0),
 (181765, 'alabama white barbecue bbq sauce', 52476, '2006-08-15', 5.0),
 (181765, 'alabama white barbecue bbq sauce', 37722, '2